In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchaudio
from df import enhance, init_df
from os.path import join as pjoin
import matplotlib.pyplot as plt
import os
import tqdm
import time
from IPython.display import Audio #listen: ipd.Audio(real.detach().cpu().numpy(), rate=FS)
import numpy as np
import scipy
import scipy.signal as sig
import pandas as pd
import torchmetrics.audio as M
from speechmos import dnsmos
from datetime import datetime

def plot_tensor(x):
    plt.plot(x.cpu().detach().numpy())

def load_audio(apath):
    audio, fs = torchaudio.load(apath)
    if fs != FS:
        #print('resampling')
        resampler = torchaudio.transforms.Resample(fs, FS)
        audio = resampler(audio)    
    if len(audio.shape) > 1:
            audio = audio[0,:]
    return audio

def power(signal):
    return np.mean(signal**2)

/home/ubuntu/enric/venvs/dns/lib/python3.8/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [2]:
FS = 48000
DURATION = 4 #time in seconds of the eval chunk
TRAINRIR_NAMES = {'D01_sb_none_NH_mono': 'singleband' , 'D02_mb_none_NH_mono': 'multiband', 
            'D03_mb_rec_NH_left': 'recdirectivity', 'D05_mb_srcrec_NH_left': 'recsourcedirectivity',
            'D00_DNS5': 'DNS5'}

use_gpu = True
if torch.cuda.is_available() and use_gpu:
    TORCH_DEVICE = "cuda"
else:
    TORCH_DEVICE = "cpu"

batch_size = 1
num_workers = 8
reverberant_noises = True

In [3]:
mix_main_path = '/home/ubuntu/Data/DNS-Challenge/V5_dev_testset/Track1_Headset/noisy'

In [4]:
mix_paths = os.listdir(mix_main_path)

In [5]:
mix_paths = [x for x in mix_paths if 'Primary' in x ]

In [6]:
mix_paths.sort()

In [7]:
mix_paths = [pjoin(mix_main_path, x) for x in mix_paths]

In [9]:
with open('DNS5_val_headset.txt', 'w') as file:
    # Write each item on a new line
    for item in mix_paths:
        file.write(f"{item}\n")